In [133]:
import os
import json
import time
import random

import io
import os.path
import re
import tarfile
import sys
import gensim
import pandas as pd
import numpy as np

from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from nltk import pos_tag


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

from nltk.tokenize import word_tokenize


In [6]:
repos_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

gutenberg_corpus_analysis_repo = os.path.join(repos_path, 'gutenberg_corpus_analysis')

In [8]:
gutenberg_repo_path = os.path.join(repos_path, 'gutenberg')
gutenberg_analysis_repo = os.path.join(repos_path, 'gutenberg-analysis')

src_dir = os.path.join(gutenberg_analysis_repo,'src')
sys.path.append(src_dir)
from data_io import get_book


gutenberg_src_dir = os.path.join(gutenberg_repo_path,'src')
sys.path.append(gutenberg_src_dir)

from metaquery import meta_query

sys.path.append(gutenberg_corpus_analysis_repo)
import misc_utils.dataset_filtering as dataset_filtering

In [10]:
text_fold=os.path.join(gutenberg_repo_path, 'data', 'text')

In [12]:
dataset='nikita_dataset'

In [14]:
train_csv = os.path.join(gutenberg_corpus_analysis_repo, dataset, 'final_train.csv')
test_csv = os.path.join(gutenberg_corpus_analysis_repo, dataset, 'final_test.csv')
val_csv = os.path.join(gutenberg_corpus_analysis_repo, dataset, 'final_val.csv')

pg_catalog_filepath=os.path.join(gutenberg_repo_path, 'metadata', 'pg_catalog.csv')

In [16]:
np.random.seed(500)

In [18]:
train_df = pd.read_csv(train_csv, index_col='Unnamed: 0')
test_df = pd.read_csv(test_csv, index_col='Unnamed: 0')
val_df = pd.read_csv(val_csv, index_col='Unnamed: 0')

train_df.head()

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects
2439,PG12810,"Uncle Sam's Boys with Pershing's Troops: Or, D...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],78,"{'World War, 1914-1918 -- Juvenile fiction', '..."
2446,PG12819,"Dick Prescott's Second Year at West Point: Or,...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],94,{'United States Military Academy -- Juvenile f...
25920,PG40605,"The Motor Boat Club at Nantucket; or, The Myst...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],189,"{'Motorboats -- Juvenile fiction', 'Nantucket ..."
55435,PG8153,"The Young Engineers in Arizona; or, Laying Tra...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],190,"{'Civil engineers -- Fiction', 'Arizona -- Fic..."
32899,PG48863,"The Motor Boat Club off Long Island; or, A Dar...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],85,"{'Motorboats -- Juvenile fiction', 'Long Islan..."


In [20]:
len(train_df['author'].unique())

80

In [22]:
subj = train_df['subjects'].replace('set()',np.nan)
subj_docs = []
for h in subj:
    try:
        h = h.strip("{}")[1:-1]
    except AttributeError:
        subj_docs.append(h)
        continue
    h = h.replace(' -- ', '-')
    h = h.replace("', '","_")
    h = h.split('_')
    h = [item.replace(' ','').replace(',', ' ') for item in h]
    h = ' '.join(h)
    subj_docs.append(h)

In [24]:
train_df['subj_str']=subj_docs

In [26]:
#train_df['subject_str'] = train_df['subjects'].apply(lambda x: split_subjects(x))

In [28]:
#train_df = train_df.sample(500)

In [30]:
start = time.time()

train_df['text'] = train_df['id'].apply(lambda x: get_book(x, path_gutenberg=gutenberg_repo_path,level='text'))
test_df['text'] = test_df['id'].apply(lambda x: get_book(x, path_gutenberg=gutenberg_repo_path,level='text'))
val_df['text'] = val_df['id'].apply(lambda x: get_book(x, path_gutenberg=gutenberg_repo_path,level='text'))

end = time.time()
print(f'Time elapsed: {end-start} seconds')

Time elapsed: 5.5778257846832275 seconds


In [32]:
def skip_start_and_end(text):
    text = text.split(' ')
    text = text[50:-50]
    return ' '.join(text)

train_df['text'] = train_df['text'].apply(skip_start_and_end)
test_df['text'] = test_df['text'].apply(skip_start_and_end)
val_df['text'] = val_df['text'].apply(skip_start_and_end)


In [34]:
def make_random_chunks(text, num_chunks=10, chunk_size=1000, overlap=False):
    chunk = []
    words = text.split(' ')

    if num_chunks * chunk_size > len(words):
        return text
    for i in range(num_chunks):
        new_words = []
        num_words = len(words)
        if chunk_size > num_words:
            chunk = chunk + words
            words = []
            return ' '.join(chunk)
        start = random.randint(0, num_words)
        chunk = [*chunk,  *words[start:start+chunk_size]]
        #print(chunk)
        if start == 0:
            words = words[chunk_size:]
        elif start == num_words - chunk_size:
            words = words[0:start]
        else:
            words = words[0:start] + words[start+chunk_size:]
    return ' '.join(chunk)

In [42]:
train_df['chunks'] = train_df['text'].apply(lambda x: make_random_chunks(x, num_chunks=10, chunk_size = 1000, overlap=False))
test_df['chunks'] = test_df['text'].apply(lambda x: make_random_chunks(x, num_chunks=10, chunk_size = 1000, overlap=False))
val_df['chunks'] = val_df['text'].apply(lambda x: make_random_chunks(x, num_chunks=10, chunk_size = 1000, overlap=False))

In [44]:
start = time.time()
with mp.Pool(11) as pool:
    train_df['tokenized'] = pool.map(word_tokenize, train_df['chunks'])
end = time.time()
print(f'Took {end-start} seconds')


Took 12.205879211425781 seconds


In [50]:
import multiprocessing as mp
start = time.time()
with mp.Pool(11) as pool:
    val_df['tokenized'] = pool.map(word_tokenize, val_df['chunks'])
end = time.time()
print(f'Took {end-start} seconds')



Took 3.855489492416382 seconds


In [51]:
import multiprocessing as mp
start = time.time()
with mp.Pool(11) as pool:
    test_df['tokenized'] = pool.map(word_tokenize, test_df['chunks'])
end = time.time()
print(f'Took {end-start} seconds')



Took 4.1067750453948975 seconds


In [54]:
outfile=os.path.join(gutenberg_corpus_analysis_repo, 'tokenized', 'train_df_chunks_tokenized.pkl')
train_df.to_pickle(outfile)

outfile=os.path.join(gutenberg_corpus_analysis_repo, 'tokenized', 'val_df_chunks_tokenized.pkl')
val_df.to_pickle(outfile)

outfile=os.path.join(gutenberg_corpus_analysis_repo, 'tokenized', 'test_df_chunks_tokenized.pkl')
test_df.to_pickle(outfile)



In [55]:
train_df = pd.read_pickle(outfile)

In [56]:
train_df[train_df['tokenized'].isnull()]

,Unnamed: 0.1,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,text,chunks,tokenized
Unnamed: 0,,,,,,,,,,,,


In [57]:
test_df[test_df['tokenized'].isnull()]

,Unnamed: 0.1,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,text,chunks,tokenized
Unnamed: 0,,,,,,,,,,,,


In [58]:
val_df[val_df['tokenized'].isnull()]

,Unnamed: 0.1,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,text,chunks,tokenized
Unnamed: 0,,,,,,,,,,,,


In [59]:
train_df['author'].unique()

array(['Doyle, Arthur Conan', 'Shakespeare, William', 'Bennett, Arnold',
       'Library of Congress. Copyright Office',
       'Haggard, H. Rider (Henry Rider)', 'Roosevelt, Theodore',
       'Jacobs, W. W. (William Wymark)', 'Howells, William Dean',
       'Hancock, H. Irving (Harrie Irving)', 'Stratemeyer, Edward',
       'Garis, Howard Roger', 'Balzac, Honoré de',
       'Chesterton, G. K. (Gilbert Keith)', 'Dumas, Alexandre',
       'Tolstoy, Leo, graf', 'Sinclair, Upton', 'Stevenson, Robert Louis',
       'Bower, B. M.', 'Wells, Carolyn', 'MacDonald, George',
       'Verne, Jules', 'Scott, Walter', 'London, Jack',
       'Wells, H. G. (Herbert George)', 'Davis, Richard Harding',
       'Kipling, Rudyard', 'James, Henry', 'Wharton, Edith',
       'Harte, Bret', 'Yonge, Charlotte M. (Charlotte Mary)',
       'Ellis, Edward Sylvester', 'Hawthorne, Nathaniel',
       'Garrett, Randall', 'Lang, Andrew', 'Twain, Mark',
       'Lever, Charles James', 'Trollope, Anthony',
       'Goldfra

In [78]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV                

In [80]:
def lemmatize_text(tokenized_text):
    # Declaring Empty List to store the words that follow the rules for this step
    final_words = []
    # Initializing WordNetLemmatizer()
    word_lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(tokenized_text):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_final = word_lemmatized.lemmatize(word,tag_map[tag[0]])
            final_words.append(word_final)
    return str(final_words)
    # The final processed set of words for each iteration will be stored in 'text_final'

In [106]:
start = time.time()
with mp.Pool(11) as pool:
    lemmatized = pool.map(lemmatize_text, train_df['tokenized'])
end = time.time()
print(f'Took {end-start} seconds')
train_df['lemmatized'] = lemmatized

Took 37.59381318092346 seconds


In [107]:
import multiprocessing as mp
start = time.time()
with mp.Pool(11) as pool:
    lemmatized = pool.map(lemmatize_text, val_df['tokenized'])
end = time.time()
print(f'Took {end-start} seconds')
val_df['lemmatized'] = lemmatized

Took 37.27150344848633 seconds


In [108]:
import multiprocessing as mp
start = time.time()
with mp.Pool(11) as pool:
    lemmatized = pool.map(lemmatize_text, test_df['tokenized'])
end = time.time()
print(f'Took {end-start} seconds')
test_df['lemmatized'] = lemmatized

Took 37.826552391052246 seconds


In [117]:
train_df.shape

(240, 13)

In [115]:
Train_X, Train_Y = train_df['lemmatized'], train_df['author']
Test_X, Test_Y = test_df['lemmatized'], test_df['author']
Val_X, Val_Y = val_df['lemmatized'], val_df['author']

In [ ]:
# Train_X, _, Train_Y, _ = model_selection.train_test_split(train_df['lemmatized'], train_df['author'],test_size=0.3)
# Val_X, _, Val_Y, _ = model_selection.train_test_split(val_df['lemmatized'], val_df['author'],test_size=0.3)
# Test_X, _, Test_Y, _ = model_selection.train_test_split(test_df['lemmatized'], test_df['author'],test_size=0.3)


In [119]:
Encoder = LabelEncoder()
Train_Y_e = Encoder.fit_transform(Train_Y)
Test_Y_e = Encoder.fit_transform(Test_Y)
Val_Y_e = Encoder.fit_transform(Val_Y)


In [122]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train_df['lemmatized'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
Val_X_Tfidf = Tfidf_vect.transform(Val_X)


In [136]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC()
SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset

predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
predictions_SVM_train = SVM.predict(Train_X_Tfidf)# Use accuracy_score function to get the accuracy
predictions_SVM_val = SVM.predict(Val_X_Tfidf)# Use accuracy_score function to get the accuracy


print("SVM Accuracy Score: Training Data -> ",accuracy_score(predictions_SVM_train, Train_Y)*100)
print("SVM Accuracy Score: Test Data -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM Accuracy Score: Validation Data -> ",accuracy_score(predictions_SVM_val, Val_Y)*100)

SVM Accuracy Score: Training Data ->  100.0
SVM Accuracy Score: Test Data ->  100.0
SVM Accuracy Score: Validation Data ->  37.083333333333336


In [148]:
# Evaluate
print("Classification Report:\n")
train_class_rpt = classification_report(Train_Y, predictions_SVM_train)
print(train_class_rpt)

Classification Report:

                                          precision    recall  f1-score   support

                             A. L. O. E.       1.00      1.00      1.00         3
                         Aimard, Gustave       1.00      1.00      1.00         3
                     Alger, Horatio, Jr.       1.00      1.00      1.00         3
                       Balzac, Honoré de       1.00      1.00      1.00         3
                    Barbour, Ralph Henry       1.00      1.00      1.00         3
               Baring-Gould, S. (Sabine)       1.00      1.00      1.00         3
            Baum, L. Frank (Lyman Frank)       1.00      1.00      1.00         3
                         Belloc, Hilaire       1.00      1.00      1.00         3
                         Bennett, Arnold       1.00      1.00      1.00         3
         Benson, E. F. (Edward Frederic)       1.00      1.00      1.00         3
                     Blanchard, Amy Ella       1.00      1.00      1.00  

In [150]:

print("Validation Classification Report:\n")
test_class_rpt = classification_report(Val_Y, predictions_SVM_val)
print(test_class_rpt)

Validation Classification Report:

                                          precision    recall  f1-score   support

                             A. L. O. E.       0.00      0.00      0.00         3
                         Aimard, Gustave       1.00      0.67      0.80         3
                     Alger, Horatio, Jr.       0.00      0.00      0.00         3
                       Balzac, Honoré de       1.00      0.33      0.50         3
                    Barbour, Ralph Henry       0.50      0.33      0.40         3
               Baring-Gould, S. (Sabine)       0.25      0.33      0.29         3
            Baum, L. Frank (Lyman Frank)       0.00      0.00      0.00         3
                         Belloc, Hilaire       0.00      0.00      0.00         3
                         Bennett, Arnold       0.00      0.00      0.00         3
         Benson, E. F. (Edward Frederic)       0.50      0.33      0.40         3
                     Blanchard, Amy Ella       0.00      0.00 

/home/dean/anaconda3/envs/school-env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dean/anaconda3/envs/school-env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dean/anaconda3/envs/school-env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap